In [3]:
import pandas as pd
from pathlib import Path


In [4]:
path = str(Path().absolute().parent)
data_dir = path + "/data"
aggregated_results_loc = data_dir + "/aggregated_results/20_21/predictions_and_results_log.csv"
print(aggregated_results_loc)

/Users/benjamin/PycharmProjects/APPLE/data/aggregated_results/20_21/predictions_and_results_log.csv


In [5]:
# load in aggregated results

In [6]:
aggregated_results = pd.read_csv(aggregated_results_loc)

In [7]:
display(aggregated_results)

,Date,Time,Week,HomeTeam,AwayTeam,APPLE Prediction,DD Prediction,JR Prediction,LC Prediction,MW Prediction,FTR
0,09/12/2020,12:30:00,1,Fulham,Arsenal,A,A,A,A,A,A
1,09/12/2020,15:00:00,1,Crystal Palace,Southampton,A,D,D,H,A,H
2,09/12/2020,20:00:00,1,West Ham,Newcastle,H,A,A,H,A,A
3,09/12/2020,17:30:00,1,Liverpool,Leeds,H,H,H,H,H,H
4,13/09/2020,14:00:00,1,West Brom,Leicester,H,D,A,A,A,H
5,13/09/2020,16:30:00,1,Tottenham,Everton,H,A,H,H,D,A
6,14/09/2020,20:00:00,1,Brighton,Chelsea,A,D,A,A,A,A
7,19/09/2020,12:30:00,2,Everton,West Brom,H,H,H,H,H,H
8,19/09/2020,15:00:00,2,Leeds,Fulham,H,H,H,H,H,H
9,19/09/2020,17:30:00,2,Man United,Crystal Palace,H,H,H,H,H,A


## Box Plots

In [8]:
import plotly.express as px

In [9]:
df = px.data.tips()
print(df)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [10]:
def correct_pred(col1, col2):
    if col1 == col2:
        return 1
    else:
        return 0

# take the weeks present 
weeks_present = set(aggregated_results["Week"].to_list())
# names of the predictors

user_prediction_cols = []
aggregated_results_cols = aggregated_results.columns.to_list()

for col in aggregated_results_cols:
    if "Prediction" in col:
        user_prediction_cols.append(col)

weekly_accuracy_df = pd.DataFrame(columns=['Predictor','Correct Predictions','Week','Accuracy of Predictions (%)'])

# filter for each week
for week in weeks_present:
    filtered_aggregated_results = aggregated_results[aggregated_results["Week"] == week].reset_index()
    no_matches = filtered_aggregated_results.shape[0]
    predictors = []
    for col in user_prediction_cols:
        predictors.append(col.strip(" Prediction"))
        filtered_aggregated_results[col.strip(" Prediction")] = filtered_aggregated_results.apply(lambda x: correct_pred(x[col], x['FTR']), axis=1)
    correct_res = filtered_aggregated_results.sum()
    correct_res = correct_res[predictors]
    summed_results_df = pd.DataFrame({'Predictor': correct_res.index, 'Correct Predictions': correct_res.values})
    summed_results_df["Week"] = week
    summed_results_df['Accuracy of Predictions (%)'] = (summed_results_df['Correct Predictions'] / no_matches) * 100
    weekly_accuracy_df = pd.concat([weekly_accuracy_df, summed_results_df], ignore_index=True)

print(weekly_accuracy_df)

  Predictor Correct Predictions Week Accuracy of Predictions (%)
0     APPLE                   4    1                     57.1429
1        DD                   4    1                     57.1429
2        JR                   4    1                     57.1429
3        LC                   4    1                     57.1429
4        MW                   4    1                     57.1429
5     APPLE                   7    2                          70
6        DD                   6    2                          60
7        JR                   7    2                          70
8        LC                   8    2                          80
9        MW                   7    2                          70


In [11]:
fig = px.box(weekly_accuracy_df, y="Accuracy of Predictions (%)")
fig.show()

NameError: name 'weekly_summed' is not defined